# make a new function to make a 2D slice based on 1D Earth model (or 2D or 3D)

In [1]:
# Nobuaki Fuji Oct 2025

# breaking news! confirmed by Lorette and Stéphanie, the CMB is ellipsoidal! 
# when working with geodetic earth models, r_PREM should be corrected as a function of latitude!

using Pkg


cd(@__DIR__)
Pkg.activate("../..")
ParamFile = "../test/testparam.csv"
include("../src/batchRevise.jl") 

myInclude("../src/batchDrWatson.jl")
myInclude("../src/DSM1D.jl")
#myInclude("../src/GeoPoints.jl")
using .DSM1D
includet("../src/GeoPoints_hesa.jl")
using .GeoPoints_hesa


  Activating project at `~/Documents/FUJI/Github/flexibleDSM`
┌ Info: Including with Revise: ../src/batchDrWatson.jl
└ @ Main /Users/hessiemohammadi/Documents/FUJI/Github/flexibleDSM/OPTmotors/src/batchRevise.jl:25
┌ Info: Including with Revise: ../src/DSM1D.jl
└ @ Main /Users/hessiemohammadi/Documents/FUJI/Github/flexibleDSM/OPTmotors/src/batchRevise.jl:25


  0.006108 seconds (2.17 k allocations: 107.062 KiB, 74.45% compilation time)


In [2]:
using Geodesy, Interpolations, StaticArrays, GMT, LinearAlgebra

In [3]:
import Base: +,-,/,*
struct GeoPoint
    lat::Float64 # in degree
    lon::Float64 # in degree
    alt::Float64 # in metre
    ecef::SVector{3,Float64}
    radius::Float64 # in metre
    #effectiveRadius::Float64 # in metre (useful to get values from 1D averaged model)
end

struct localCoord2D
    ix::Int64
    iz::Int64
    x::Float64
    z::Float64
    horizontalVector::SVector{2,Float64}
    normalVector::SVector{2,Float64}
end

struct localCoord3D
    ix::Int64
    iy::Int64
    iz::Int64
    x::Float64
    y::Float64
    z::Float64
    horizontalVector1::SVector{3,Float64}
    horizontalVector2::SVector{3,Float64}
    normalVector::SVector{3,Float64}
end

function localCoord2D(ix,iz,x,z)
    localCoord2D(ix,iz,x,z,(0.0,0.0),(0.0,0.0))
end

function GeoPoint(lat::Float64, lon::Float64; alt=0.0, ell=wgs84)
    lla = LLA(lat,lon, alt) # be careful LLA uses degrees by default!!
    ecef_coords = ECEF(lla,ell)
    radius = norm([ecef_coords.x,ecef_coords.y,ecef_coords.z])
    GeoPoint(lat, lon, alt, SVector(ecef_coords.x, ecef_coords.y, ecef_coords.z),radius)
end


function GeoPoint(ecef::SVector{3,Float64}; ell=wgs84)
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function +(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef + b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function -(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef - b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function /(a::GeoPoint,c::Real; ell=wgs84)
    ecef=a.ecef / c
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end


function *(a::GeoPoint,c::Real; ell=wgs84)
    ecef=a.ecef * c
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end


function effectiveRadius(a::GeoPoint,r0::Float64; ell=wgs84)
    radiusPlanetHere = GeoPoint(a.lat,a.lon).radius 
    ratio = r0/radiusPlanetHere
    return a.radius*ratio
end




effectiveRadius (generic function with 1 method)

In [4]:
# Example 1-D model (toy). Replace with your actual model arrays.
# r_model in meters from Earth center (0..R_ref), assume sorted ascending
R_ref = DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3                   # model reference radius (m)


6.371e6

In [5]:
# create an interpolator in radius space (ensure interpolation domain covers used r)
#r_model = collect(0.0:1000.0:R_ref)  # example radius grid
#v_model = 5000 .- 0.5 .* (R_ref .- r_model) ./ 1000;  # dummy profile
#itp = LinearInterpolation(r_model, v_model; extrapolation_bc=Flat())

In [6]:
include("../src/GeoPoints_hesa.jl")
using .GeoPoints_hesa

#p1 = GeoPoint(48.8566,2.3522) # Paris
p1 = GeoPoint(35.3606,138.7274) # Mt. FUJI
#p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)
p2 = GeoPoint(52.71,153.43) # Kuril
#topographyFromGeoPoint(p1)
@show GeoPoint(p1.ecef)
@show p0 = (p1 + p2)/2.0

x, y, z, R = makeLocalCoordinates(p1, p2)

println("Midpoint distance (km): ", distance_km(p1, p2))

GeoPoint(p1.ecef) = GeoPoint(35.3606, 138.7274, 7.041648957460953e-10, [-3.9137588253302076e6, 3.435006364207505e6, 3.6705657776570977e6], 6.371014832277791e6)


p0 = (p1 + p2) / 2.0 = GeoPoint(44.27390988756807, 144.99202054057008, -99017.88272902492, [-3.6885964274361087e6, 2.583548665106721e6, 4.360811114004266e6], 6.268743166017594e6)


UndefVarError: UndefVarError: `makeLocalCoordinates` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
# pre-definition of x-axis vector p1->p2 (normally this is not the one we want)
p2_1 = p2-p1
x_axis_tentative = (p2_1/p2_1.radius).ecef


3-element SVector{3, Float64} with indices SOneTo(3):
  0.20122105373601673
 -0.7609228584663001
  0.6168520819425368

In [8]:
# definition of z-axis vector centre -> p0

z_axis = normalize(p0.ecef)

3-element SVector{3, Float64} with indices SOneTo(3):
 -0.5884108392622822
  0.4121318415327577
  0.6956436080590939

In [9]:
# y-axis: complete right-handed system
y_axis = normalize(cross(z_axis, x_axis_tentative))

3-element SVector{3, Float64} with indices SOneTo(3):
 0.7835587838754027
 0.5029426942721885
 0.3648071798771668

In [10]:
x_axis = cross(y_axis, z_axis)  # now perfectly orthogonal

3-element SVector{3, Float64} with indices SOneTo(3):
  0.19952021564331823
 -0.759734158421911
  0.6188664573852697

In [11]:
#Rotation matrix
R = SMatrix{3,3,Float64}(
    x_axis[1], x_axis[2], x_axis[3],
    y_axis[1], y_axis[2], y_axis[3],
    z_axis[1], z_axis[2], z_axis[3]
)

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
  0.19952   0.783559  -0.588411
 -0.759734  0.502943   0.412132
  0.618866  0.364807   0.695644

In [12]:
R*R'

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 1.0         1.0252e-17  1.9449e-17
 1.0252e-17  1.0         1.1695e-16
 1.9449e-17  1.1695e-16  1.0

In [13]:
p_2D_to_ECEF(x_2D,z_2D,pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = pOrigin+R*SVector(x_2D,0.e0,z_2D)

p_2D_to_ECEF (generic function with 1 method)

In [14]:
p_ECEF_to_2D(p_3D::SVector{3,Float64},pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = R' * (p_3D - pOrigin)

p_ECEF_to_2D (generic function with 1 method)

In [15]:
p_ECEF_to_2D(p0.ecef, p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
     1.118975634672664e6
     3.183999215672772e-10
 -3236.068810317997

In [16]:
p0.ecef - p_2D_to_ECEF(p_ECEF_to_2D(p0.ecef,p1.ecef,R)[1],p_ECEF_to_2D(p0.ecef,p1.ecef,R)[3],p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
 4.656612873077393e-10
 0.0
 9.313225746154785e-10

In [17]:
p0_0_above=p_2D_to_ECEF(0,0,p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
 -3.9137588253302076e6
  3.435006364207505e6
  3.6705657776570977e6

In [18]:
p0_1m_above=p_2D_to_ECEF(0,1,p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
 -3.913759413741047e6
  3.4350067763393465e6
  3.6705664733007057e6

In [19]:
p_ECEF_to_2D(p0_1m_above, p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
 -2.4899031362141985e-10
 -4.790273080702922e-11
  1.0000000001511575

In [20]:
Δx = 50 # in metre
Δz = 50

altMax = 20000 # in metre
altMin = -100.e3 # in metre
#altMin = -5000

# since p1 and p2 can be even out of the solid Earth or inside the Earth
# I define that p1 and p2 should be the left and right limits

leftLimit = 0 # in metre so x is always non-negative
rightLimit = p2_1.radius # it is bigger than what we need ....|


2.2379606280115275e6

In [21]:
@show Nx = Int64((rightLimit-leftLimit) ÷ Δx+1) 
@show Nz = Int64((altMax-altMin) ÷ Δz + 1 ) 

Nx = Int64((rightLimit - leftLimit) ÷ Δx + 1) = 44760
Nz = Int64((altMax - altMin) ÷ Δz + 1) = 2401


2401

In [22]:
allGridsInGeoPoints=Array{GeoPoint,2}(undef,Nx,Nz)

effectiveRadii=Array{Float64,2}(undef,Nx,Nz)

allGridsInCartesian2D=Array{localCoord2D,2}(undef,Nx,Nz)

seismicModel2D=(ρ=zeros(Float64,Nx,Nz),Vpv=zeros(Float64,Nx,Nz),Vph=zeros(Float64,Nx,Nz),Vsv=zeros(Float64,Nx,Nz),Vsh=zeros(Float64,Nx,Nz),Qμ=zeros(Float64,Nx,Nz),Qκ=zeros(Float64,Nx,Nz),QμPower=zeros(Float64,Nx,Nz),QκPower=zeros(Float64,Nx,Nz),η=zeros(Float64,Nx,Nz))

for iXZ in CartesianIndices(allGridsInGeoPoints)
    ix, iz = Tuple(iXZ)
    x = leftLimit+(ix-1)*Δx
    z = altMin+(iz-1)*Δz 
    tmpGeoPoint=GeoPoint(p_2D_to_ECEF(x,z,p1.ecef,R))
    
    allGridsInGeoPoints[iXZ]=tmpGeoPoint

    allGridsInCartesian2D[iXZ]=localCoord2D(ix,iz,x,z)
  
    effectiveRadii[iXZ]=effectiveRadius(tmpGeoPoint,DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3 )
end


In [23]:
DSM1D.my1DDSMmodel.C_Qκ

12×4 Matrix{Float64}:
  1327.7  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0
 57823.0  0.0  0.0  0.0

# about effectiveRadii corrections

In fact, there are three types of parameter modification, based on 1D planet models. 

(i) Changing effectiveRadii by the function effectiveRadius, considering the ellipticity of the planet (which is done above).

(ii) changing a topography of discontinuity (CMB, 410 or 660-km, solid-air surface), which can be further be done by changing effectiveRadii locally.

(iii) introducing a new layer (ocean or something that could exist only at some region in terms of lat and lon).


Reading the topo file can give the solid-air surface (option ii) and solid-liquid surface (option iii) so we need to separate the two of them.


The idea is to read topo for (ii) and (iii) and provide a interpolator for effectiveRadii first. Then (iii) can be done by overwriting the params matrix.

In [24]:
# now I need to use GMT.jl to get the topography

lats = [p.lat for p in allGridsInGeoPoints]
lons = [p.lon for p in allGridsInGeoPoints]

lat_min, lat_max = extrema(lats)
lon_min, lon_max = extrema(lons)
if lat_min === lat_max
    if lat_min > 1.0
        lat_min = lat_min - 0.5
    else
        lat_max = lat_max + 0.5
    end
end

if lon_min === lon_max
    if lon_min > 1.0
        lon_min = lon_min - 0.5
    else
        lon_max = lon_max + 0.5
    end
end

lat_min, lat_max,lon_min,lon_max

(35.219529455006295, 52.83260406678321, 138.63994895423002, 153.57841257897653)

In [26]:
function GMTprecision(requiredResolutionInKmÀPeuPrès::Float64)
    if requiredResolutionInKmÀPeuPrès > 55.0
        return "@earth_relief_30m"
    elseif  18.0 < requiredResolutionInKmÀPeuPrès <= 55.0
        return "@earth_relief_10m"
    elseif  9.0 < requiredResolutionInKmÀPeuPrès <= 18.0
        return "@earth_relief_05m"
    elseif 3.6 < requiredResolutionInKmÀPeuPrès <= 9.0
        return "@earth_relief_02m"
    elseif 1.8 < requiredResolutionInKmÀPeuPrès <= 3.6
        return "@earth_relief_01m"
    elseif 0.45 < requiredResolutionInKmÀPeuPrès <= 1.8
        return "@earth_relief_15s"
    elseif 0.30 < requiredResolutionInKmÀPeuPrès <= 0.45
        return "@earth_relief_10s"
    elseif 0.15 < requiredResolutionInKmÀPeuPrès <= 0.30
        return "@earth_relief_02s"
    elseif 0.09 < requiredResolutionInKmÀPeuPrès <= 0.15
        return "@earth_relief_03s"
    elseif 0.06 < requiredResolutionInKmÀPeuPrès <= 0.09
        return "@earth_relief_02s"
    else
        return "@earth_relief_01s"
        # after this
        """
        # Load your own local DEM (GeoTIFF, etc.)
        topo = GMT.read("Copernicus_DSM_30m_Europe.tif")

        # Cut a region and maybe downsample
        region = [-5, 0, 43, 47]
        topo_sub = GMT.grdcut(topo, region=region)

        # Optional resampling (to ~10 m)
        topo_10m = GMT.grdsample(topo_sub, inc="10m")

        """
    end
end

                  

GMTprecision (generic function with 1 method)

In [2]:


using GMT,CairoMakie

#region = [lon_min, lon_max, lat_min, lat_max]
region = [-10,20,35,60]
precision = GMTprecision(0.5) # this should be in Km



# resolution can be @earth_relief_01m (≈2 km), @earth_relief_03m, or coarser like 10m, 30m, etc.
topoEuro = GMT.grdcut(precision, region=region)

topoEuro_surface = copy(topoEuro.z)
topoEuro_bathymetry = copy(topoEuro.z)


ERROR: LoadError: UndefVarError: `ComputePipeline` not defined in `Makie`
Suggestion: check for spelling errors or missing imports.
Stacktrace:
 [1] include
   @ ./Base.jl:557 [inlined]
 [2] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base ./loading.jl:2881
 [3] top-level scope
   @ stdin:6
in expression starting at /Users/hessiemohammadi/.julia/packages/CairoMakie/LRBPV/src/CairoMakie.jl:1
in expression starting at stdin:6


ErrorException: Failed to precompile CairoMakie [13f3f980-e62b-5c42-98c6-ff1f3baf88f0] to "/Users/hessiemohammadi/.julia/compiled/v1.11/CairoMakie/jl_zdbD4Q".

In [ ]:
topoEuro_surface[topoEuro_surface .< 0.0 ] .= 0.0
topoEuro_bathymetry[topoEuro_bathymetry .> 0.0 ] .= 0.0

In [ ]:


fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    topoEuro.x,topoEuro.y,topoEuro_surface';
    colormap = :terrain,
    colorrange=(-1000,1000),
    axis = (aspect = DataAspect(), xlabel = "Longitude", ylabel = "Latitude", title = "Topography (m)")
)
Colorbar(fig[1,2], hm, label="Elevation (m)")
fig

In [ ]:


fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    topoEuro.x,topoEuro.y,topoEuro_bathymetry';
    colormap = :terrain,
    colorrange=(-3000,0),
    axis = (aspect = DataAspect(), xlabel = "Longitude", ylabel = "Latitude", title = "Topography (m)")
)
Colorbar(fig[1,2], hm, label="Elevation (m)")
fig

In [ ]:
using GMT,CairoMakie

region = [lon_min, lon_max, lat_min, lat_max]

precision = GMTprecision(0.03)
# resolution can be @earth_relief_01m (≈2 km), @earth_relief_03m, or coarser like 10m, 30m, etc.
topo = GMT.grdcut(precision, region=region)
topo_surface = copy(topo.z)
topo_bathymetry = copy(topo.z)
fig, ax, hm = heatmap(
    topo.x,topo.y,topo.z';
    colormap = :terrain,
    colorrange=(0,800),
    axis = (aspect = DataAspect(), xlabel = "Longitude", ylabel = "Latitude", title = "Topography (m)")
)
Colorbar(fig[1,2], hm, label="Elevation (m)")
fig

In [ ]:
topo.y

In [ ]:
# First we take care of topo_surface

topoInterpolater = interpolate((topo.y,topo.x),topo_surface,Gridded(Linear()));

In [ ]:
eps = 100.0 # in metre "below" option should be enough but who knows
for iXZ in CartesianIndices(allGridsInGeoPoints)
    tmpPoint = allGridsInGeoPoints[iXZ]
    if 0.0 <= tmpPoint.alt <= topoInterpolater(tmpPoint.lat,tmpPoint.lon) 
        # it might be very time-consuming if we do this for 3D Cartesian points ...
        effectiveRadii[iXZ]=DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3 - eps
    end
end

# note

in order to remove eps above, I need to understand interpolate.jl bit more (in the presence of discontinuities)

In [ ]:
# make a regular grid for the necessary radii
NradiusNodes =500 # I don't know how to make this number reasonable
tmpNradiusNodes = NradiusNodes
if NradiusNodes === 1
    tmpNradiusNodes = 1
end

ΔradiusIncrementInKm = (maximum(effectiveRadii)-minimum(effectiveRadii))/(tmpNradiusNodes-1) *1.e-3
linearRadiiInKm =(collect(1:1:NradiusNodes) .- 1)*ΔradiusIncrementInKm .+ minimum(effectiveRadii)*1.e-3

push!(linearRadiiInKm, DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer) # here I put the most important discontinuity but maybe I need to add all vmin inside, the order will be sorted so we don't


In [ ]:
newRadii,params=DSM1D.compute1DseismicParamtersFromPolynomialCoefficientsWithGivenRadiiArray(DSM1D.my1DDSMmodel,linearRadiiInKm,"below")

In [ ]:
itp_ρ = LinearInterpolation(newRadii.*1.e3,params.ρ; extrapolation_bc=Flat())
# I know that I don't have to copy this for all the parameters, Hesaneh, you can find an elegant way to do this for vpv, vph etc. with one line!

In [ ]:
#seismicModel2D=(ρ=zeros(Float64,Nx,Nz),Vpv=zeros(Float64,Nx,Nz),Vph=zeros(Float64,Nx,Nz),Vsv=zeros(Float64,Nx,Nz),Vsh=zeros(Float64,Nx,Nz),Qμ=zeros(Float64,Nx,Nz),Qκ=zeros(Float64,Nx,Nz),QμPower=zeros(Float64,Nx,Nz),QκPower=zeros(Float64,Nx,Nz),η=zeros(Float64,Nx,Nz))
for iXZ in CartesianIndices(allGridsInGeoPoints)
    seismicModel2D.ρ[iXZ]= itp_ρ(effectiveRadii[iXZ])
end

In [ ]:
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel2D.ρ;
    colormap = :seismic,
    colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "density model")
)
Colorbar(fig[1,2], hm, label="density")
#ylims!(ax,-50,50)
#xlims!(ax,600,700)
fig

# I need to do (iii) for the water

In [ ]:
radii,params=DSM1D.compute1DseismicParamtersFromPolynomialCoefficientsWithGivenRadiiArray(DSM1D.my1DDSMmodel,myRadiiArray,"below")
params.Vph

In [ ]:
allGridsInGeoPoints[1,1]

In [ ]:
effectiveRadii[200,1]

In [ ]:
params.ρ

In [ ]:
# resolution can be @earth_relief_01m (≈2 km), @earth_relief_03m, or coarser like 10m, 30m, etc.

# I need to : 
add local coordinates and normals at each point as options to GeoPoint


In [ ]:

# Extract coordinates from GeoPoints
lons = [p.lon for p in allGridsInGeoPoints[:]]
lats = [p.lat for p in allGridsInGeoPoints[:]]


In [ ]:

# Query the topography via grdtrack using the file path
topo_vals = GMT.grdtrack(lons, lats; G=tmpfile,show=false)

In [ ]:


# Reshape back to 2D
Nx, Nz = size(allGridsInGeoPoints)
topo_vals_2D = reshape(topo_vals, Nx, Nz)
